# Limit 

## Set Up

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import re
import json
import pickle
import pandas as pd
from pathlib import Path

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import sys
# Add the workspace root to sys.path so 'scripts' can be imported
sys.path.append(str(Path.cwd().parents[3]))

from scripts.feature_extractions.gigahorse import get_n_gram_from_gigahorse_assessment


In [3]:
PATH = Path.cwd().parents[3]
MODEL_PATH = os.path.join(PATH, 'models/baseline')
DATA_PATH = os.path.join(PATH, 'data')
EX_PATH = os.path.join(DATA_PATH, 'external/crpwarner/large')
HEX_PATH = os.path.join(EX_PATH, 'hex')

## Load and Prepare Data

In [4]:
df = pd.read_excel(os.path.join(EX_PATH, 'sample/limit.xlsx'))
df['Address'] = df['Address'].str.lower()
df = df.set_index('Address')
df.head()

,TP?
Address,
0xe5f3c6d2b47cbe2cf936b9521466bac2422ebef8,Yes
0xa623b5a542c0d7daadef321042a04c600b03a8cb,Yes
0xe412189da2dfa188a1a61633114b8732bbbfba19,Yes
0x9fcf7acdc11fd904c4b73a009909c7f00efc4844,Yes
0x921a5dce3dfed5cccfbb2e593f2978533bc66110,Yes


In [5]:
df.isna().sum()

TP?    0
dtype: int64

In [ ]:
files = []
addresses = []
y = []

for path in list(Path(HEX_PATH).glob('*.hex')):
    address = path.stem.lower()
    filename = path.name
    if address in df.index:
        addresses.append(address)
        files.append(filename)
        tp_value = df.loc[address, 'TP?']
        if isinstance(tp_value, pd.Series):
            tp_value = tp_value.iloc[0]
        y.append(1 if tp_value == 'Yes' else 0)


X = get_n_gram_from_gigahorse_assessment (EX_PATH, addresses, files)

[*] Creating container...
[*] Starting analysis...


Compiling /opt/RugPull/gigahorse/logic/main.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/clientlib/function_inliner.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/logic/fallback_scalable.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/logic/fallback_precise.dl to C++ program and executable
Processing contract names.
Setting up workers.
Analysing...

Using scalable fallback decompilation configuration for 0x1f4a9567c1f98a8c9d7f026827f099b41a2e54d6.hex
0: 0x1f4a9567c1f98a8c9d7f026827f099b41a completed in 0.02 + 0.44 + 0.43 + 0.00 secs.

Finished 1 contracts...



--------------------------------------------------------------------------------
Analytics
--------------------------------------------------------------------------------



Writing results to results.json


Hidden Mint Function:  False
Leaking Token:  False
Limiting Sell Order:  False
[*] Copying results.json...
[*] Copying all .facts files...
[+] results.json saved to: /Users/napatcholthaipanich/Dev/master/dissertation/workspace/ml/data/external/crpwarner/large/out/0x1f4a9567c1f98a8c9d7f026827f099b41a2e54d6.result.json
[✓] 0x1f4a9567c1f98a8c9d7f026827f099b41a2e54d6 completed
[*] Creating container...
[*] Starting analysis...


Compiling /opt/RugPull/gigahorse/logic/fallback_scalable.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/logic/main.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/clientlib/function_inliner.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/logic/fallback_precise.dl to C++ program and executable
Processing contract names.
Setting up workers.
Analysing...

Using scalable fallback decompilation configuration for 0x76bc9e61a1904b82cbf70d1fd9c0f8a120483bbb.hex
0: 0x76bc9e61a1904b82cbf70d1fd9c0f8a120 completed in 0.02 + 0.85 + 0.74 + 0.00 secs.

Finished 1 contracts...



--------------------------------------------------------------------------------
Analytics
--------------------------------------------------------------------------------



Writing results to results.json


Hidden Mint Function:  False
Leaking Token:  False
Limiting Sell Order:  False
[*] Copying results.json...
[*] Copying all .facts files...
[+] results.json saved to: /Users/napatcholthaipanich/Dev/master/dissertation/workspace/ml/data/external/crpwarner/large/out/0x76bc9e61a1904b82cbf70d1fd9c0f8a120483bbb.result.json
[✓] 0x76bc9e61a1904b82cbf70d1fd9c0f8a120483bbb completed
[*] Creating container...
[*] Starting analysis...


...

Hidden Mint Function:  False
Leaking Token:  False
Limiting Sell Order:  False
[*] Copying results.json...
[*] Copying all .facts files...
[+] results.json saved to: /Users/napatcholthaipanich/Dev/master/dissertation/workspace/ml/data/external/crpwarner/large/out/0x3f1db2c3e98232971ec8ccfea8be865228763fef.result.json
[✓] 0x3f1db2c3e98232971ec8ccfea8be865228763fef completed
[*] Creating container...
[*] Starting analysis...


Compiling /opt/RugPull/gigahorse/logic/main.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/clientlib/function_inliner.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/logic/fallback_scalable.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/logic/fallback_precise.dl to C++ program and executable
Processing contract names.
Setting up workers.
Analysing...

Using scalable fallback decompilation configuration for 0x1a93f35557700cdeab1205673126fea1a5fcd814.hex
0: 0x1a93f35557700cdeab1205673126fea1a5 completed in 0.03 + 0.75 + 0.75 + 0.00 secs.

Finished 1 contracts...



--------------------------------------------------------------------------------
Analytics
--------------------------------------------------------------------------------



Writing results to results.json


Hidden Mint Function:  False
Leaking Token:  False
Limiting Sell Order:  False
[*] Copying results.json...
[*] Copying all .facts files...
[+] results.json saved to: /Users/napatcholthaipanich/Dev/master/dissertation/workspace/ml/data/external/crpwarner/large/out/0x1a93f35557700cdeab1205673126fea1a5fcd814.result.json
[✓] 0x1a93f35557700cdeab1205673126fea1a5fcd814 completed
[*] Creating container...
[*] Starting analysis...


Compiling /opt/RugPull/gigahorse/logic/main.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/logic/fallback_scalable.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/clientlib/function_inliner.dl to C++ program and executable
Compiling /opt/RugPull/gigahorse/logic/fallback_precise.dl to C++ program and executable
context canceled


KeyboardInterrupt: 

In [ ]:
X

In [ ]:
y[:5]

In [ ]:
with open(os.path.join(DATA_PATH, 'processed/byte_freq/features.json'), 'r', encoding='utf-8') as f:
    train_cols = json.load(f)

In [ ]:
X = X.reindex(columns=train_cols, fill_value=0).reset_index(drop=True)
X.head()

## Load Model and Evaluate

In [ ]:
model_paths = list(Path(MODEL_PATH).glob('leak-*.pkl'))

In [ ]:
def get_best_model_path(paths):
    best_f1_score = -1.0
    best_path = None

    for path in paths:
        # Extract the F1 score using a regular expression
        match = re.search(r'f1-(\d+\.\d+)\.(pkl|json|keras)$', path.name)
        if match:
            try:
                f1_score = float(match.group(1))
                if f1_score > best_f1_score:
                    best_f1_score = f1_score
                    best_path = path
            except ValueError:
                # Handle cases where the extracted string might not be a valid float
                print(f"Warning: Could not parse F1 score from filename: {path.name}")
        else:
            print(f"Warning: F1 score pattern not found in filename: {path.name}")

    return best_path

In [ ]:
best_model_path = get_best_model_path(model_paths)

if best_model_path:
    model = pickle.load(open(best_model_path, 'rb'))

In [ ]:
y_pred = model.predict(X)

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
cm = confusion_matrix(y, y_pred)
cm_display = ConfusionMatrixDisplay(cm)
cm_display.plot(cmap=plt.cm.Blues,values_format='g')
plt.show()